In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
url = 'https://www.imdb.com/feature/genre/?ref_=nv_ch_gr'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
ge = [i.get('title') for i in soup.find_all('img')]

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By

list=[]
for genre_p in ge:
    driver = webdriver.Chrome()
    driver.get(url)
    driver.maximize_window()
    driver.implicitly_wait(20)
    action = driver.find_element(By.XPATH, '//img[@title="'+genre_p+'"]')
    action.click()
    soup = BeautifulSoup(driver.page_source, "html.parser")
    while True:
        movies = soup.find_all('div', class_='lister-item-content')
        titles = [i.h3.a.text for i in movies]
        years = [i.select('span[class="lister-item-year text-muted unbold"]')[0].text for i in movies]
        certificate = [i.select('span.certificate') for i in movies]
        certificate = ["" if i==[] else i[0].text for i in certificate ]
        genre = [''.join(i.select('span.genre')[0].text.split()) for i in movies]
        runtime = [i.select('span.runtime') for i in movies]
        runtime = ["" if i==[] else i[0].text for i in runtime ]
        rating = [i.select('div[class="inline-block ratings-imdb-rating"] > strong') for i in movies]
        rating = ["" if i==[] else i[0].text for i in rating ]
        metascore = [i.select('div[class="inline-block ratings-metascore"] > span') for i in movies]
        metascore = ["" if i==[] else ''.join(i[0].text.split()) for i in metascore ]
        description = [i.select('p[class="text-muted"]')[1].text[1:] for i in movies]
        votes = [i.select('p[class="sort-num_votes-visible"] > span[name="nv"]') for i in movies]
        votes = ["" if i==[] else i[0].get('data-value') for i in votes ]
        gross = [i.select('p[class="sort-num_votes-visible"] > span[name="nv"]') for i in movies]
        gross = ["" if len(i)<2 else i[1].get('data-value') for i in gross ]
        texts = ["" if i==[] else ''.join(i.select('p[class=""]')[0].text.split()) for i in movies]
        Director=[]
        Stars=[]
        for i in texts:
            if 'Stars:' in i:
                index=i.index('Stars:')
                Stars.append(i[index+6:])
                Director.append("") if index==0 else Director.append(i[9:index-1])
            else:
                if 'Director:' in i:
                    Stars.append('')
                    Director.append(i[9:])
                else:
                    Stars.append('')
                    Director.append('')
        try:
            for index in range(0, len(movies)):
                data = {"movie_title": titles[index],
                        "year": years[index],
                        "certificate": certificate[index],
                        "runtime": runtime[index],
                        "genre": genre[index],
                        "ratings": rating[index],
                        "metascore": metascore[index],
                        "description": description[index],
                        "Director": Director[index],
                        "stars": Stars[index],
                        "votes": votes[index],
                        "gross": gross[index]
                        }
                list.append(data)
        except:
            print(titles[0])
        print(len(list))
        next_url = soup.select_one('a[class="lister-page-next next-page"]').get('href')
        try:
            soup = BeautifulSoup(requests.get("https://www.imdb.com"+next_url).text, "html.parser")
        except:
            break
driver.quit()

50
100
150
200
250
300
350
400
450
500
550
600


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B9DCE3+50899]
	(No symbol) [0x00B2E111]
	(No symbol) [0x00A35588]
	(No symbol) [0x00A1D333]
	(No symbol) [0x00A7F4DB]
	(No symbol) [0x00A8DB33]
	(No symbol) [0x00A7B6F6]
	(No symbol) [0x00A57708]
	(No symbol) [0x00A5886D]
	GetHandleVerifier [0x00E03EAE+2566302]
	GetHandleVerifier [0x00E392B1+2784417]
	GetHandleVerifier [0x00E3327C+2759788]
	GetHandleVerifier [0x00C35740+672048]
	(No symbol) [0x00B38872]
	(No symbol) [0x00B341C8]
	(No symbol) [0x00B342AB]
	(No symbol) [0x00B271B7]
	BaseThreadInitThunk [0x761B6B89+25]
	RtlGetFullPathName_UEx [0x77738F9F+1215]
	RtlGetFullPathName_UEx [0x77738F6D+1165]


In [12]:
df = pd.DataFrame(list)
df.to_csv('data_3.csv',index=False)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283650 entries, 0 to 283649
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   movie_title  283650 non-null  object
 1   year         283650 non-null  object
 2   certificate  283650 non-null  object
 3   runtime      283650 non-null  object
 4   genre        283650 non-null  object
 5   ratings      283650 non-null  object
 6   metascore    283650 non-null  object
 7   description  283650 non-null  object
 8   Director     283650 non-null  object
 9   stars        283650 non-null  object
 10  votes        283650 non-null  object
 11  gross        283650 non-null  object
dtypes: object(12)
memory usage: 26.0+ MB
